In [1]:
import os
from gzip import GzipFile
from itertools import product
from typing import AnyStr, Optional

import orjson as json
import pandas as pd
from price_parser.parser import parse_price

# Parse results into Pandas

In [2]:
with GzipFile("Results ëC4.json.gz", "rb") as fh:
    results = json.loads(fh.read())

In [3]:
def parse_car_info(
    info: dict[AnyStr, AnyStr], page_number: int, car_number: int
) -> Optional[pd.DataFrame]:
    if "Exception" in info:
        return None

    dicts = {k: d for k, d in info.items() if type(d) == dict}
    lists = {k: d for k, d in info.items() if type(d) != dict}

    df = pd.concat(
        (pd.DataFrame.from_dict(v, orient="index").T for v in dicts.values()),
        keys=[k for k in dicts.keys()],
        axis="columns",
    )

    for k, v in lists.items():
        df[k] = v

    df["PAGE #"] = page_number
    df["CAR #"] = car_number

    return df

In [4]:
df = (
    pd.concat(
        filter(
            lambda df: df is not None,
            (
                df := parse_car_info(
                    results[p]["CARS"][c]["CHARACTERISTICS"], p + 1, c + 1
                )
                for p, c in product(range(39), range(9))
            ),
        ),
        axis="index",
    )
    .set_index(["PAGE #", "CAR #"])
    .rename_axis(columns=["Section", "Title"])
    .sort_index()
)


In [5]:
# df.index = df.index.to_flat_index()
# df.columns = df.columns.to_flat_index()

In [6]:
# df.head()

# Clean results

In [7]:
idx = pd.IndexSlice

In [8]:
cdf = df.drop(
    columns=[
        "AUTRES",
        "CARBURANT / ENERGIE",
        "CONSOMMATION WLTP",
        "DÉPOLLUTION / NORME",
        "DIMENSIONS DU VÉHICULE (EN MM)",
        "EQUIPEMENTS ADDITIONNELS",
        "JANTES",
        "MASSES",
        "MONTANT PRIX BATTERIE",
        "MOTORISATION",
        "PERFORMANCES",
        "PNEUMATIQUES",
        "VALIDITÉ DE LA FICHE",
        "VOLUMES INTÉRIEURS (EN M3 NORME VDA)",
    ],
    level="Section",
).reindex()

In [9]:
duplicated_columns = [c for c in cdf.columns if cdf[c].nunique(dropna=False) == 1]
cdf = cdf.drop(columns=duplicated_columns)

In [10]:
boolean_columns = [c for c in cdf.columns if c[1]]
cdf = cdf.astype({bc: "boolean" for bc in boolean_columns})
cdf[boolean_columns] = cdf[boolean_columns].fillna(False)


In [11]:
cdf.loc[:, ["FINITION"]] = cdf["FINITION"].astype("category")
cdf["PRICE"] = (
    cdf["PRICE"].apply(lambda p: parse_price(p).amount_float).astype("UInt32")
)
cdf["VALIDITY"] = pd.to_datetime(cdf["VALIDITY"])

# Filter results

In [12]:
features = [v for v in cdf.columns.get_level_values("Title").sort_values() if v]
with open("Features ëC4.txt", "w") as fh:
    for feature in features:
        fh.write(feature + os.linesep)

In [13]:
pd.concat(
    (
        cdf.loc[:, idx["FINITION", :]],
        cdf.loc[:, idx[:, "Recharge sans fil pour smartphone"]],
    ),
    axis="columns",
).groupby("FINITION").sum()


Section,SYSTÈMES AUDIO ET TÉLÉMATIQUE
Title,Recharge sans fil pour smartphone
FINITION,
FEEL,0
FEEL PACK,0
SHINE,0
SHINE PACK,131


In [14]:
pd.concat(
    (
        cdf.loc[:, idx["FINITION", :]],
        cdf.loc[:, idx[:, "Affichage tête haute couleur"]],
    ),
    axis="columns",
).groupby("FINITION").sum()


Section,SÉCURITÉ
Title,Affichage tête haute couleur
FINITION,
FEEL,0
FEEL PACK,0
SHINE,132
SHINE PACK,134


In [15]:
pd.concat(
    (
        cdf.loc[:, idx["FINITION", :]],
        cdf.loc[:, idx[:, "Système audio numérique (DAB) MP3 6 HP"]],
    ),
    axis="columns",
).groupby("FINITION").sum()


/tmp/ipykernel_4692/1631595051.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ).groupby("FINITION").sum()


Section,EQUIPEMENTS
Title,Système audio numérique (DAB) MP3 6 HP
FINITION,
FEEL,14
FEEL PACK,69
SHINE,129
SHINE PACK,0


In [16]:
pd.concat(
    (
        cdf.loc[:, idx["FINITION", :]],
        cdf.loc[:, idx[:, "Sièges AV réglables hauteur + Lombaire conducteur"]],
    ),
    axis="columns",
).groupby("FINITION").sum()


/tmp/ipykernel_4692/3850005438.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ).groupby("FINITION").sum()


Section,CONFORT ET FONCTIONNALITÉ
Title,Sièges AV réglables hauteur + Lombaire conducteur
FINITION,
FEEL,13
FEEL PACK,69
SHINE,125
SHINE PACK,0


In [17]:
pd.concat(
    (
        cdf.loc[:, idx["FINITION", :]],
        cdf.loc[:, idx[:, "Sièges avant chauffants"]],
    ),
    axis="columns",
).groupby("FINITION").sum()


/tmp/ipykernel_4692/2173765916.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ).groupby("FINITION").sum()


Section,CONFORT ET FONCTIONNALITÉ
Title,Sièges avant chauffants
FINITION,
FEEL,0
FEEL PACK,0
SHINE,0
SHINE PACK,112


In [18]:
pd.concat(
    (
        cdf.loc[:, idx["FINITION", :]],
        cdf.loc[:, idx[:, "Volant chauffant"]],
    ),
    axis="columns",
).groupby("FINITION").sum()


/tmp/ipykernel_4692/3978699740.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  ).groupby("FINITION").sum()


Section,CONFORT ET FONCTIONNALITÉ
Title,Volant chauffant
FINITION,
FEEL,0
FEEL PACK,0
SHINE,132
SHINE PACK,134
